# Human Infrastructure Data & EDA


1. Data Provenance 

    a. Where is the data from?


    
    b. Is it reliable? (partially subjective)
    
    
2. Time measures

    a. Time range of the dataset
        2014-01-01T00:00:00Z - 2022-05-01T00:00:00
    
    b. Granularity (Weekly, Monthly, Aperiodic, Daily, Timestamp)
         
    
    
3. Location

    a. Is it available for the UK, Brazil/India?
 
    
    b. Granularity (is it a pin point location?, Rounded coordinates?...)
    
    c. Format (shape files, lat long, geographical code)
    
    
4. Data Gaps and size

    a. Size of the data
        Entire world is around 250TB. We only need monthly composites for some regions ans some years
        
    b. Available columns and columns of interest
 

    c. Missing data (amount and years/locations missing)
    
    d. Nulls or Nans
    
    e. Primary key
    
    f. Columns to join on
    
    g. Any additional fields

    
    
5. Ease of access

    a. Is there a free to use license?s
 
    
    b. How is data accessed? (API, download)
    
    c. Is the documentation straightforward?
        
    
    d. Any limits on how much data we can download?
       


https://ghsl.jrc.ec.europa.eu/
https://www.openrailwaymap.org/
Roadways map/bridges https://developers.google.com/maps/documentation/javascript/trafficlayer
Electrical lines? (is this important)
Dams


In [8]:
import pandas as pd
import geemap, ee
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

try:
        ee.Initialize()
except Exception as e:
        ee.Authenticate()
        ee.Initialize()
        

In [15]:
# get our Nepal boundary
aoi = ee.FeatureCollection("FAO/GAUL/2015/level0").filter(ee.Filter.eq('ADM0_NAME','U.K. of Great Britain and Northern Ireland')).geometry()

ghsl = ee.ImageCollection('JRC/GHSL/P2016/SMOD_POP_GLOBE_V1').filter(ee.Filter.date('2015-01-01', '2015-12-31')).select('smod_code').median();

# create a boolean mask setting anything equaling 2 (low density) or 3 (high density) as True
# this will actually be our binary label layer
#ghslbinary = ghsl.gte(2)

ghslVis= {"min":0.0, "max":3.0,"palette":['000000', '448564', '70daa4', 'ffffff']}
#ghslbiVis= {"palette":['000000', 'ffffff']}

map1 = geemap.Map()
map1.centerObject(aoi,7)
map1.addLayer(ghsl.clip(aoi), ghslVis, 'Degree of Urbanization')
#map1.addLayer(ghslbinary.clip(aoi), ghslbiVis, 'Built up')
map1.addLayerControl()
map1

Map(center=[54.03949956479122, -2.7881851686945835], controls=(WidgetControl(options=['position', 'transparent…

In [20]:
center_lat = -3.497341
center_lon = 38.642394


aoi = ee.Geometry.Point([lon, lat]).buffer(15000);
ghsl_clipped = ghsl.clip(aoi)
ghslVis= {"min":0.0, "max":3.0,"palette":['000000', '448564', '70daa4', 'ffffff']}

map2 = geemap.Map(center=[lat, lon],zoom=10)
map2.add_basemap('SATELLITE')
map2.addLayer(ghsl_clipped, ghslVis, "Degree of urbanization")
map2.addLayerControl()
map2

Map(center=[12.9716, 77.5946], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(chi…

In [21]:
def get_avg_deg_urban(lat, lon, buffer=10000, scale=100, geometry = None):
    """
    buffer (int): value in meters of radius around center lat lon
    """
    ghsl = ee.ImageCollection('JRC/GHSL/P2016/SMOD_POP_GLOBE_V1').\
    filter(ee.Filter.date('2015-01-01', '2015-12-31')).select('smod_code').median()

    aoi = ee.Geometry.Point([lon, lat]).buffer(buffer);
    ghsl_clipped = ghsl.clip(aoi)
    
    smod_code = ghsl_clipped.reduceRegion(reducer=ee.Reducer.mean(),
                                     geometry = None,
                                     scale=scale,
                                     maxPixels=1e9).get('smod_code')
    
    return smod_code.getInfo()

In [22]:
lat = 12.9716
lon = 77.5946
get_avg_deg_urban(lat, lon)

2.9949516291458917